[Reference](https://medium.com/geekculture/building-a-data-visualization-web-app-using-python-streamlit-c1efd1bed7a)

In [1]:
pip install streamlit

     |████████████████████████████████| 9.7 MB 9.0 MB/s 
     |████████████████████████████████| 164 kB 72.9 MB/s 
     |████████████████████████████████| 4.3 MB 46.1 MB/s 
     |████████████████████████████████| 180 kB 58.7 MB/s 
     |████████████████████████████████| 76 kB 5.9 MB/s 
     |████████████████████████████████| 111 kB 74.6 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 128 kB 45.6 MB/s 
     |████████████████████████████████| 792 kB 49.0 MB/s 
     |████████████████████████████████| 380 kB 42.6 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=1343c4d4a1289c4888474c47f03e6ff67cfba8059df765716047b6b1394f2b22
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      S

In [1]:
!streamlit hello

2022-02-15 12:48:07.945 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  Welcome to Streamlit. Check out our demo in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.247.106.5:8501

  Ready to create your own Python apps super quickly?
  Head over to https://docs.streamlit.io

  May you create awesome apps!



  Stopping...
^C


In [2]:
import requests as r
import pandas as pd

def get_data(url):
  api = r.get(url)
  api_status_code = api.status_code #200

  api_data = api.json() #converting into json format
  api_data_normal = pd.json_normalize(api_data) #converting into dataframe
  
  return api_data_normal

get_data('https://api.tfl.gov.uk/AccidentStats/2019')

,$type,id,lat,lon,location,date,severity,borough,casualties,vehicles
0,Tfl.Api.Presentation.Entities.AccidentStats.Ac...,348291,51.552111,-0.300335,On High Road Near The Junction With Thurlow Road,2019-02-12T13:16:00Z,Slight,Brent,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
1,Tfl.Api.Presentation.Entities.AccidentStats.Ac...,348292,51.445068,0.153080,On Bourne Road Near The Junction With southwol...,2019-02-12T12:20:00Z,Serious,Bexley,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
2,Tfl.Api.Presentation.Entities.AccidentStats.Ac...,348293,51.585638,0.054139,On Lodge Hill Near The Junction With Woodford ...,2019-01-21T08:45:00Z,Slight,Redbridge,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
3,Tfl.Api.Presentation.Entities.AccidentStats.Ac...,348294,51.465243,-0.378282,On Staines Road Near The Junction With Welling...,2019-02-07T13:00:00Z,Slight,Hounslow,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
4,Tfl.Api.Presentation.Entities.AccidentStats.Ac...,348295,51.516526,-0.128906,On Earnshaw Street Near The Junction With New ...,2019-02-13T10:00:00Z,Slight,Camden,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
...,...,...,...,...,...,...,...,...,...,...
50621,Tfl.Api.Presentation.Entities.AccidentStats.Ac...,726846,51.511451,0.011427,On Silvertown Way Near The Junction With Georg...,2019-06-04T18:47:00Z,Slight,Newham,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
50622,Tfl.Api.Presentation.Entities.AccidentStats.Ac...,726847,51.510148,-0.414286,On Coldharbour Lane Near The Junction With Min...,2019-04-20T18:01:00Z,Slight,Hillingdon,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
50623,Tfl.Api.Presentation.Entities.AccidentStats.Ac...,726848,51.501774,-0.168485,Location Uncertain Rutland Gate,2019-06-05T13:40:00Z,Slight,City of Westminster,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...
50624,Tfl.Api.Presentation.Entities.AccidentStats.Ac...,726849,51.537787,-0.167819,Location Uncertain Avenue Road,2019-06-05T09:40:00Z,Slight,Camden,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...,[{'$type': 'Tfl.Api.Presentation.Entities.Acci...


In [5]:
api_data_normal = get_data('https://api.tfl.gov.uk/AccidentStats/2019')

api_casualities = pd.json_normalize(api_data_normal['casualties'])
api_casualities_col  = pd.json_normalize(api_casualities[0])
#api_casualities_col['$type'][0]
api_casualities_col = api_casualities_col.drop(columns = '$type')
casuality_cols = api_casualities_col.columns
age = pd.Series(api_casualities_col['age'], name = 'age')
casualities_class = pd.Series(api_casualities_col['class'], name = 'class')
severity = pd.Series(api_casualities_col['severity'], name = 'severity')
mode = pd.Series(api_casualities_col['mode'], name = 'mode')
ageBand = pd.Series(api_casualities_col['ageBand'], name = 'ageBand')

api_vehicles = pd.json_normalize(api_data_normal['vehicles'])
api_vehicles_col = pd.json_normalize(api_vehicles[0])
api_vehicles_col = pd.Series(api_vehicles_col['type'], name = 'vehicles')

api_data_normal = api_data_normal.drop(columns = ['$type','casualties', 'vehicles'])
api_data_modified = pd.concat([api_data_normal,age, casualities_class, mode, ageBand, api_vehicles_col], axis=1)


class_dict = dict(api_data_modified['class'].value_counts())
class_df = pd.DataFrame(class_dict.items(), columns = ['class', 'count'])

mode_dict = dict(api_data_modified['mode'].value_counts())
mode_df = pd.DataFrame(mode_dict.items(), columns = ['mode', 'count'])

age_dict = dict(api_data_modified['age'].value_counts())
age_df = pd.DataFrame(age_dict.items(), columns = ['age', 'count'])

drop_cols = ['id', 'location', 'date', 'severity', 'borough', 'age', 'class', 'mode', 'ageBand', 'vehicles']
map_df = api_data_modified.drop(columns = drop_cols)

In [6]:
map_df

,lat,lon
0,51.511963,-0.028211
1,51.371636,-0.117621
2,51.514951,-0.072747
3,51.519173,-0.262356
4,51.565743,-0.136308
...,...,...
50621,51.398872,-0.088527
50622,51.485776,-0.150157
50623,51.528556,-0.332035
50624,51.512031,-0.449557


In [7]:
import streamlit as st

dataExploration = st.container()

with dataExploration:
  st.title('Transport for London')
  st.subheader('Keeping London moving...')
  st.header('Dataset: Transport for London')
  st.markdown('I found this dataset at... https://tfl.gov.uk/info-for/open-data-users/')
  st.markdown('**Basically, it is a "London AccidentStats" dataset for the year 2019**')
  st.text('Below is the sample DataFrame')
  st.write(api_data_modified.head())

In [8]:
map_con = st.container()

drop_cols = ['id', 'location', 'date', 'severity', 'borough', 'age', 'class', 'mode', 'ageBand', 'vehicles']
map_df = api_data_modified.drop(columns = drop_cols)

with map_con:
  st.header('A simple map of the accident zones in london')
  st.map(map_df, use_container_width =  True)

In [9]:
@st.cache
def get_data():
  api = r.get('https://api.tfl.gov.uk/AccidentStats/2019')
  api_status_code = api.status_code

  api_data = api.json()
  api_data_normal = pd.json_normalize(api_data)
  cols = api_data_normal.columns
  return api_data_normal